# Finding steady states of genetic circuit from rates

There are 4 types of rates: creation / degration, and coupling / decoupling (aka association / dissociation). From these, steady states should be able to be worked out.

## Base rates

The creation and degadation rate of RNAs can be worked out from the known concentration of RNA in a cell. Since we are using E. coli, the concentration of RNA should be in the range of 75-120 mg/ml ([from bionumbers](https://bionumbers.hms.harvard.edu/bionumber.aspx?s=n&v=2&id=100042#:~:text=In%20E.,%2C%20respectively%20(primary%20sources).)). Since E. coli have a cell volume of 1.1 $\mu m^3$ (ranging from 0.44-1.79) ([from bionumbers](https://bionumbers.hms.harvard.edu/bionumber.aspx?&id=100004&ver=20)), which is approximately 1.1e-12 ml, this gives the following amount of RNA per cell:

$$ 1.1e^{-12} ml * 75 mg/ml = 8.25e^{-11} mg $$
$$ 1.1e^{-12} ml * 120 mg/ml = 1.32e^{-10} mg $$

The molecular weight of RNA can vary depending on the length and nucleotide makup of the RNA, but can be calculated approximately as $(nucleotides * 320.5) + 159.0$ to get the molecular weight in Daltons. While the RNA we are considering for simplified genetic circuits are very short in length (in order to remain within a manageable combinatorial sequence space), we will use the average length of nucleotides for RNA in E. coli, which is about 1000 nts ([from bionumbers](https://bionumbers.hms.harvard.edu/bionumber.aspx?id=100022)). This gives a molecular weight of 320659 Da or g/mol ([from ThermoFisher](https://www.thermofisher.com/uk/en/home/references/ambion-tech-support/rna-tools-and-calculators/dna-and-rna-molecular-weights-and-conversions.html)), which is equal to $1.88 * 10^{12}$ molecules.

$$ \frac{ 8.25e^{-11} mg }{ 320659 g/mol * 1000mg / g } = 2.5728266e-19 mol $$ 
$$ 2.5728266e-19 mol * 6.02214179e23 molecules / mol = 154939.265863 molecules $$ 

The steady state average amount of RNA in E. coli is about 1380 molecules of mRNA per cell ([from bionumbers](https://bionumbers.hms.harvard.edu/bionumber.aspx?s=n&v=3&id=111919)), with a degradation rate half life of ca. 5-10 minutes ([from "Quantifying E. coli proteome and transcriptome with single-molecule sensitivity in single cells" (2010)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2922915/)). 
RNA transcription occurs at a rate of approximately 39-55 nucleotides per second in E. coli ([from bionumbers](https://bionumbers.hms.harvard.edu/bionumber.aspx?&id=104900)). Since the RNA being considered for simplified genetic circuits are very short in length (in order to remain within a manageable combinatorial sequence space), we assume a nucleotide length of 20 per RNA molecule This would make the removal rate the following:

$$ \frac{dRNA}{dt} = k_{prod} - k_{deg}*RNA $$
$$ \overline{RNA} = \frac{k_{prod}}{k_{deg}} $$
$$ k_{prod} = \frac{39 nt/s}{20 nt/molecule} = 1.95 molecules/s $$
$$ k_{deg} = \frac{k_{prod}}{\overline{RNA}} = \frac{1.95 molecules/s}{1380 molecules} = 0.00141 s^{-1} $$

In [ ]:

a = 0.08333
d = 0.0008333
dRNA = 